In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal/research'

In [3]:
os.chdir('../.')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    score: Path    

In [6]:
from src.constants import *
from src.utils.common import read_yaml,save_json,create_directories

2024-09-22 00:09:14,814 - INFO - start of looging


In [7]:
class ConfigurationManagerone:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.Model_Evaluation
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            score = config.score
        )
        return model_evaluation_config

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from src.components.data_aug import DataAugmentation
from src.entity import DataAugConfig
from src.config.configuration import ConfigurationManager

In [9]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig,evl_config:DataAugmentation):
        self.config = config
        self.data_config = evl_config

    def model_load(self):
        model = load_model(self.config.model_path)
        return model

    def model_evaluate(self):
        model = self.model_load()
        obj = DataAugmentation(self.data_config)
        train_data, test_data, val_data = obj.augment_data()

        self.train_acc = model.evaluate(train_data, verbose=0)[1]
        self.test_acc = model.evaluate(test_data, verbose=0)[1]
        
        print(f"Training Accuracy: {self.train_acc * 100:.2f}%")
        print(f"Testing Accuracy: {self.test_acc * 100:.2f}%")

    def save_score(self):
        scores = {
            "train_acc": self.train_acc,
            "test_acc": self.test_acc
        }
        json_path = self.config.score
        save_json(path = Path(json_path), data = scores)

In [10]:
try:
    config = ConfigurationManagerone()
    eval_config = config.get_model_evaluation_config()

    data_confog = ConfigurationManager()
    aug_config = data_confog.get_data_aug()

    model_eval = ModelEvaluation(config = eval_config, evl_config = aug_config)
    model_eval.model_evaluate()
    model_eval.save_score()
except Exception as e:
    raise e

2024-09-22 00:09:17,333 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-22 00:09:17,336 - INFO - Yaml file: params.yaml loaded successfully
2024-09-22 00:09:17,336 - INFO - created directory at: data
2024-09-22 00:09:17,337 - INFO - created directory at: data/model_evaluation
2024-09-22 00:09:17,338 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-22 00:09:17,340 - INFO - Yaml file: params.yaml loaded successfully
2024-09-22 00:09:17,340 - INFO - created directory at: data
Found 272 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 58 images belonging to 2 classes.


/Applications/anaconda3/envs/chkn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Training Accuracy: 79.41%
Testing Accuracy: 93.33%
2024-09-22 00:09:54,820 - INFO - json file saved at: data/model_evaluation/chicken_score.json
